# Setup

In [ ]:
import os
from dotenv import load_dotenv

if os.getcwd().endswith('notebooks'):
    %cd -q ..

import ep_parse.core as core
import ep_parse.case_data as cdata
import ep_parse.utils as pu

load_dotenv()
pu.configure_logging()

%load_ext autoreload
%autoreload 2

case_id = "CASE002"

# Load from Ensite export
Create the heart geometries as well as MAP and RF tags from the Ensite export (and from medtronic bookmark file, if needed)

## Store epmed/ensite offset

In [ ]:
# Attempt to infer epmed/ensite offset

offset = core.infer_map_system_offset(case_id)

if offset:
    print(f"Added an offset of {offset} to the meta file for {case_id}.  Skip the cell below!")
else:
    print("Unable to infer an epmed/ensite offset.  Use the cell below to set manually")

### Set offset manually if inference failed above

In [ ]:
epmed_time = "12:00:00.000"  # "12:00:00.000" format
ensite_time = "12:00:00.100"  # time in ensite system corresponding to epmed_time above
FORCE = False   # use to override an existing value in the case meta file

#### End Configs

offset = core.store_ensite_offset(case_id, epmed_time, ensite_time, force=FORCE)

## Generate the geometry and tag files

In [ ]:
parse_types = ["geometry", "RF", "MAP"]  # subset of ["geometry", "RF", "MAP"]
#### End Configs

core.generate_tags_from_ensite(case_id, parse_types)

### Filter MAP tags by duration

In [ ]:
min_duration = 1.4  # seconds
#### End Configs

from datetime import timedelta
tags = cdata.load_case_tags(case_id)
new_tags = [t for t in tags if not t['label'].startswith("MAP") or (pu.as_datetime(t['end_time']) - pu.as_datetime(t['start_time'])) > timedelta(seconds=min_duration)]
print(f"Removed {len(tags) - len(new_tags)} tags")
cdata.write_case_tags(case_id, new_tags)

### Filter MAP tags to those with signal data

In [ ]:
tags = cdata.load_case_tags(case_id)
channels_with_data = cdata.case_channels(case_id)
new_tags = [t for t in tags if not t['label'].startswith("MAP") or set(t.get("channels", {}).keys()).intersection(channels_with_data)]
to_remove = set([t["catheter"] for t in tags if t['label'].startswith("MAP") and not len(set(t.get("channels", {}).keys()).intersection(channels_with_data))])
print(f"Removed {len(tags) - len(new_tags)} tags for catheters {to_remove}")
cdata.write_case_tags(case_id, new_tags)

# Load from CARTO export
Create the heart geometries as well as MAP and RF tags from the CARTO export

In [ ]:
parse_types = ["geometry", "RF", "MAP"]  # subset of ["geometry", "RF", "MAP"]
parse_opts = {
    "min_points_per_MAP": 2,  # if > 1, results are grouped by stability
}
#### End Configs

core.generate_tags_from_carto(case_id, parse_types, parse_opts)


## Adjust Carto Tag Times

In [ ]:
seconds_to_add = 1.5   # e.g.  -1.25, 2, 5.62352

##### End Configs

import ep_parse.nb_utils.tag_adjustment as tga
tga.add_seconds_to_tags(case_id, seconds_to_add)

# Display parsed Heart Models and Tags

In [ ]:
from ep_parse.plots.model3d_plots import plot_heart

vtk_mesh_path = f"data/{case_id}/{case_id}_LA_and_RA.vtk"

configs = {
    "mesh_color": "#f8f3f3",
    "mesh_opacity": 0.9,
    "background_color": "#3f3f3f",
    # "rf_tag_color": "",
    # "map_tag_color": "",
    # "misc_tag_color": ""

}

plot_heart(case_id, vtk_mesh_path, **configs)